# Data Generation Script for Training Clip

## Imports

In [121]:
import os
import json
import random
import string
from PIL import Image
import pytesseract
import easyocr
import numpy as np
import torch
from multiprocessing import set_start_method
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
from threading import Lock
from functions import *

## Global Variables

In [122]:
in_dir = "/mnt/nis_lab_research/data/coco_files/raw/shah_b1_539_21"
out_dir = "/mnt/nis_lab_research/data/clip_data/test"
out_res_w = 224
out_res_h = 224
bg_color = "white"
padding = 0.05

## Preprocessing

In [123]:
with open(os.path.join(in_dir, "result.json")) as f:
    obj = json.load(f)

In [124]:
img_list = obj["images"]
cat_list = obj["categories"]
ann_list = obj["annotations"]

In [125]:
cat_list

[{'id': 0, 'name': 'Accept Button'},
 {'id': 1, 'name': 'Address Input Box'},
 {'id': 2, 'name': 'Advertisement'},
 {'id': 3, 'name': 'Alert Notification'},
 {'id': 4, 'name': 'Allow Button'},
 {'id': 5, 'name': 'Checkbox'},
 {'id': 6, 'name': 'Click Captcha'},
 {'id': 7, 'name': 'Close Button'},
 {'id': 8, 'name': 'Download Button'},
 {'id': 9, 'name': 'Email Input Box'},
 {'id': 10, 'name': 'General Button'},
 {'id': 11, 'name': 'General Input Box'},
 {'id': 12, 'name': 'Image Captcha'},
 {'id': 13, 'name': 'Login Button'},
 {'id': 14, 'name': 'Logo'},
 {'id': 15, 'name': 'Name Input Box'},
 {'id': 16, 'name': 'Password Input Box'},
 {'id': 17, 'name': 'Phone Input Box'},
 {'id': 18, 'name': 'Play Button'},
 {'id': 19, 'name': 'Popup'},
 {'id': 20, 'name': 'Search Button'},
 {'id': 21, 'name': 'Search Input Box'},
 {'id': 22, 'name': 'Submit Button'},
 {'id': 23, 'name': 'Text Captcha'},
 {'id': 24, 'name': 'Toggle Button'},
 {'id': 25, 'name': 'Update Button'},
 {'id': 26, 'name': '

In [126]:
cat_map = []
for cat in cat_list:
    cat_map.append(cat["name"])
cat_map = sorted(cat_map)

In [127]:
cat_map

['Accept Button',
 'Address Input Box',
 'Advertisement',
 'Alert Notification',
 'Allow Button',
 'Checkbox',
 'Click Captcha',
 'Close Button',
 'Download Button',
 'Email Input Box',
 'General Button',
 'General Input Box',
 'Image Captcha',
 'Login Button',
 'Logo',
 'Name Input Box',
 'Password Input Box',
 'Phone Input Box',
 'Play Button',
 'Popup',
 'Search Button',
 'Search Input Box',
 'Submit Button',
 'Text Captcha',
 'Toggle Button',
 'Update Button',
 'Video']

## Functions

## Main

In [128]:
if not os.path.exists(out_dir):
        os.makedirs(out_dir)
for cat in cat_list:
    os.makedirs(os.path.join(out_dir, cat["name"]), exist_ok=True)

In [129]:
print(torch.cuda.is_available())

True


In [130]:
torch.cuda.empty_cache()

In [131]:
set_start_method('spawn', True)
print_lock = Lock()

if __name__ == "__main__":

    with ProcessPoolExecutor(max_workers=os.cpu_count()) as executor:
        
        futures = {}
        for i, img in enumerate(img_list):
            print(i)
            img_bn = os.path.basename(img["file_name"])[0:-4]
            img_fp = os.path.join(in_dir, "images", os.path.basename(img["file_name"]))
            img_id = img["id"]
            for j, ann in enumerate(ann_list):
                ann_img_id = ann["image_id"]
                cat_id = ann["category_id"]
                if img_id == ann_img_id:
                    future = executor.submit(process_image_annotation, img_fp, out_dir, img_bn, ann, cat_map, cat_id, bg_color, out_res_w, out_res_h, j)
                    futures[future] = (i, j)
                
            completed = 0
            for future in as_completed(futures):
                try:
                    result = future.result()
                    with print_lock:
                        completed += 1
                        print(f"Completed images: {completed}, {futures[future]}")
                except Exception as exc:
                    print(f"Generated an exception: {exc}")
    

0


Generated an exception: name 'out_dir' is not defined
Generated an exception: name 'out_dir' is not defined
Generated an exception: name 'out_dir' is not defined
Generated an exception: name 'out_dir' is not defined
Generated an exception: name 'out_dir' is not defined
Generated an exception: name 'out_dir' is not defined
Generated an exception: name 'out_dir' is not defined
Generated an exception: name 'out_dir' is not defined
Generated an exception: name 'out_dir' is not defined
Generated an exception: name 'out_dir' is not defined
Generated an exception: name 'out_dir' is not defined
1
Generated an exception: name 'out_dir' is not defined
Generated an exception: name 'out_dir' is not defined
Generated an exception: name 'out_dir' is not defined
Generated an exception: name 'out_dir' is not defined
Generated an exception: name 'out_dir' is not defined
Generated an exception: name 'out_dir' is not defined
Generated an exception: name 'out_dir' is not defined
Generated an exception: na

KeyboardInterrupt: 